In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d1 import MyEnv
#from test_env_for_lstm import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions

from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.read_wright_weights import save_weights
from utility.read_wright_weights import reload_weights
from utility.save_logs import save_logs
from utility.save_logs import save_hists
from utility.save_logs import save_env_info

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 1
CONTINUAL = True
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', MyRNNUAVClass)

eval_env = MyEnv()
policies_own = {
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    #"red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
    #          {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    #"red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
    #          {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
policies_enem = {
    "red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    "red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
# policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env
config_own = ppo.DEFAULT_CONFIG.copy()
config_own = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 1,
          "train_batch_size": 1200*25,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_own,  "policy_mapping_fn": policy_mapping_fn}
         }
config_enem = ppo.DEFAULT_CONFIG.copy()
config_enem = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 1,
          "train_batch_size": 600*5*10,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_enem,  "policy_mapping_fn": policy_mapping_fn}
         }

res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config_own,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

adversary = ppo.PPOTrainer(config=config_enem,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    
    #trainer.restore(checkpoint_path=model_path)
    #save_weights("blue_0",trainer)
    #save_weights("blue_1",trainer)

    #reload_weights(policy_id="red_0",trainer=trainer,set_policy_id="blue_0")
    #reload_weights(policy_id="red_1",trainer=trainer,set_policy_id="blue_1")
    reload_weights(policy_id="blue_0",trainer=trainer,set_policy_id="blue_0")
    reload_weights(policy_id="blue_1",trainer=trainer,set_policy_id="blue_1")
    #save_weights("red_0",trainer)
    #save_weights("red_1",trainer)


models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
hist_dir = os.path.join('./' + PROJECT + '/hist/')
if not os.path.exists(hist_dir):
    os.makedirs(hist_dir)

for j in range(2):
    text_name = models_dir + TRIAL + "blue_"+str(j) +'.txt'
    with open(text_name, "w") as fp:
        trainer.get_policy("blue_"+str(j)).model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
    png_name = models_dir + TRIAL + '.png'
    plot_model(trainer.get_policy("blue_"+str(j)).model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-04-06 17:31:47,879	INFO trainer.py:2141 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-04-06 17:31:47,880	INFO ppo.py:250 -- In multi-agent mode, policies will be optimized sequentially by the multi

2022-04-06 17:32:05,872	INFO trainable.py:130 -- Trainable.setup took 17.995 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-06 17:32:05,874	WARNING util.py:55 -- Install gputil for GPU system monitoring.


2022-04-06 17:32:18,910	INFO trainable.py:130 -- Trainable.setup took 12.984 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-06 17:32:18,912	WARNING util.py:55 -- Install gputil for GPU system monitoring.


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [ ]:

eval_env.reset()
save_env_info(eval_env)
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.eval = False
    eval_env.self_play = False
    eval_env.reset()
    results = trainer.train()
    save_logs(res_name,results,steps,CONTINUAL)
    print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n-------------- Evaluation at steps:{steps} starting ! --------------')

        check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
            trainer.restore(checkpoint_path=model_path)
            eval_env.eval = True
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            #fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100

            cell_size = 256
            state_0=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            state_1=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            action_dict0 = [0,0]
            action_dict1 = [0,0]
            rewards = {"blue_0":0,"blue_1":0}
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                #video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                action_dict0 = trainer.compute_single_action(obs['blue_0'],
                                                             state=state_0,prev_action=None,prev_reward=None,
                                                             policy_id='blue_0',explore=False)
                action_dict1 = trainer.compute_single_action(obs['blue_1'],
                                                             state=state_1,prev_action=None,prev_reward=None,
                                                             policy_id='blue_1',explore=False)

                state_0 = action_dict0[1]
                state_1 = action_dict1[1]
                obs, rewards, dones, infos = eval_env.step({'blue_0': action_dict0[0], 'blue_1': action_dict1[0]})

                env_blue_pos_temp_mod, env_red_pos_temp_mod, env_mrm_pos_temp_mod = render_env.copy_from_env_mod(eval_env)
                if eval_env.timer == 1:
                    env_blue_pos_mod = env_blue_pos_temp_mod
                    env_red_pos_mod = env_red_pos_temp_mod
                    env_mrm_pos_mod = env_mrm_pos_temp_mod
                else:
                    env_blue_pos_mod = np.vstack([env_blue_pos_mod,env_blue_pos_temp_mod])
                    env_red_pos_mod = np.vstack([env_red_pos_mod,env_red_pos_temp_mod])
                    env_mrm_pos_mod = np.vstack([env_mrm_pos_mod,env_mrm_pos_temp_mod])


                #if record_mode == 0:
                    #img = np.array(fig.canvas.renderer.buffer_rgba())
                    #img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    #video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()

                # エピソードの終了処理
                if dones['__all__']:
                    save_hists("blue_"+str(i),steps,env_blue_pos_mod,hist_dir)
                    save_hists("red_"+str(i),steps,env_red_pos_mod,hist_dir)
                    save_hists("mrm_"+str(i),steps,env_mrm_pos_mod,hist_dir)
                    save_weights("blue_0",trainer)
                    save_weights("blue_1",trainer)
                    # print(f'all done at {env.steps}')
                    break
                

            
            #if record_mode == 0:
               # video.release()

ray.shutdown()

-------------------------- Scene: 0 --------------------------

----------------- Training at steps:0 start! -----------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
128 blue_0 DOWN
297 blue_1 Shoot at red_0 launch distance : 59246.83754764299 True True
308 blue_1 Shoot at red_0 launch distance : 54441.71987798446 True True
342 red_1 Shoot at blue_1
353 red_1 Shoot at blue_1
410 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 410 -0.1009998613389792 -1.3054293203804836
blue_1 False False 410 0.3585499719943541 102.76600210523038
-------------------------- Scene: 0 --------------------------
328 blue_0 Shoot at red_0 launch distance : 59890.70634179611 True True
339 blue_0 Shoot at red_0 launch distance : 54270.29902315596 True True
364 red_1 Shoot at blue_0
375 red_1 Shoot at blue_0
419 blue_0: Destroyed
443 blue_0 Splash :red_0
492 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 492 0.8989998525

291 blue_0 Shoot at red_1 launch distance : 53905.234898675546 True True
307 blue_0 Shoot at red_1 launch distance : 45875.77310566843 True True
342 red_0 Shoot at blue_0
353 red_0 Shoot at blue_0
391 blue_0 Splash :red_1
408 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 408 0.8587054208994445 106.36661151952553
blue_1 False False 408 0.898999287566111 -0.30438293345892586
-------------------------- Scene: 0 --------------------------
285 blue_0 Shoot at red_0 launch distance : 58863.56646952429 True True
297 blue_0 Shoot at red_0 launch distance : 53311.9621540053 True True
333 red_1 Shoot at blue_0
344 red_1 Shoot at blue_0
396 blue_0: Destroyed
399 blue_0 Splash :red_0
TIME LIMIT LOSE
blue_0 False False 1200 0.8989991676733691 90.53551053627606
blue_1 False False 1200 0.8929991676733691 -5.879153860735327
-------------------------- Scene: 0 --------------------------
110 blue_1 DOWN
197 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 197 -1.1010000778141975 -2.237899662509223


282 blue_0 Shoot at red_1 launch distance : 58422.820062373314 True True
Same tgt shoot
Same tgt shoot
Same tgt shoot
294 blue_0 Shoot at red_0 launch distance : 59285.97078212875 True True
312 red_0 Shoot at blue_1
323 red_0 Shoot at blue_1
368 blue_1 Splash :red_1
409 blue_0 Splash :red_0
WIN
blue_0 False True 523 12.293446433588274 126.44228092706814
blue_1 False True 523 12.293455066921608 149.42244329735587
-------------------------- Scene: 0 --------------------------
297 blue_1 Shoot at red_0 launch distance : 53784.88044330612 True True
323 blue_1 Shoot at red_0 launch distance : 40597.85284854872 True True
335 red_1 Shoot at blue_1
346 red_1 Shoot at blue_1
398 blue_1 Splash :red_0
405 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 0.9939938523698663 -13.368619648741625
blue_1 False False 1200 0.8989997190365329 114.08595355746523
-------------------------- Scene: 0 --------------------------
268 blue_0 Shoot at red_1 launch distance : 59461.52637937869 True True
Sa

112 blue_1 DOWN
288 blue_0 Shoot at red_0 launch distance : 59735.15086792742 True True
300 blue_0 Shoot at red_0 launch distance : 53686.52710991031 True True
346 red_1 Shoot at blue_0
357 red_1 Shoot at blue_0
399 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 399 0.35864285527027495 88.93314048413072
blue_1 False False 399 0.39899992193694167 -0.7032862205770457
-------------------------- Scene: 0 --------------------------
285 blue_0 Shoot at red_0 launch distance : 59416.079813043834 True True
296 blue_0 Shoot at red_0 launch distance : 54141.03872167007 True True
348 red_1 Shoot at blue_0
359 red_1 Shoot at blue_0
401 blue_0 Splash :red_0
413 blue_0: Destroyed
1133 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 1133 0.8990005502389192 92.72885415987531
blue_1 False False 1133 -0.10699944976108089 -15.316197430695915
-------------------------- Scene: 0 --------------------------
278 blue_0 Shoot at red_1 launch distance : 58245.76222494474 True True
289 blue_0 Shoot at red_1

2022-04-06 18:40:56,316	WARNING deprecation.py:46 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


...

173 blue_1 DOWN
301 blue_0 Shoot at red_0 launch distance : 58095.616181330304 True True
312 blue_0 Shoot at red_0 launch distance : 53217.87419441325 True True
354 red_1 Shoot at blue_0
365 red_1 Shoot at blue_0
419 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 419 0.35869337936034096 101.91315752972808
blue_1 False False 419 0.39899991269367424 -0.9051346468313393
-------------------------- Scene: 0 --------------------------
294 blue_1 Shoot at red_0 launch distance : 57384.9012222683 True True
305 blue_1 Shoot at red_0 launch distance : 51818.500063457446 True True
345 red_1 Shoot at blue_1
356 red_1 Shoot at blue_1
404 blue_1 Splash :red_0
420 blue_1: Destroyed
995 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 995 -0.10700223717293911 -50.28225332846919
blue_1 False False 995 0.8989991961603941 119.77740212630157
-------------------------- Scene: 0 --------------------------
294 blue_1 Shoot at red_0 launch distance : 59187.89742553443 True True
305 blue_1 Shoot at red_0 l

2022-04-07 00:19:11,227	INFO trainable.py:496 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000006\checkpoint-6


agent_timesteps_total: 365028
custom_metrics: {}
date: 2022-04-07_00-19-10
done: false
episode_len_mean: 768.58
episode_media: {}
episode_reward_max: 292.7951070300288
episode_reward_mean: 91.41909993756279
episode_reward_min: -123.27718656628204
episodes_this_iter: 37
episodes_total: 260
experiment_id: 923874cad5764ae39ede4597e28bf38e
hostname: DESKTOP
info:
  learner:
    blue_0:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 6.719954967498779
        entropy_coeff: 0.0
        kl: 0.12369590252637863
        model: {}
        policy_loss: 0.044928040355443954
        total_loss: 70.336181640625
        vf_explained_var: 0.025899820029735565
        vf_loss: 70.2665023803711
      train: null
    blue_1:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 7.231827259063721
        entropy_coeff: 0.0
        kl: 0.11217445880174637
        model: {}


2022-04-07 00:19:11,228	INFO trainable.py:503 -- Current state after restoring: {'_iteration': 6, '_timesteps_total': 182514, '_time_total': 23977.971690416336, '_episodes_total': 260}


-------------------------- Scene: 0 --------------------------
279 blue_0 Shoot at red_0 launch distance : 59957.29118176192 True True
290 blue_0 Shoot at red_0 launch distance : 54331.456426168384 True True
310 red_1 Shoot at blue_1
321 red_1 Shoot at blue_1
381 blue_1: Destroyed
393 blue_0 Splash :red_0
TIME LIMIT LOSE
blue_0 False False 1200 0.9940008190154186 113.48780930665816
blue_1 False False 1200 0.8990008190154186 81.8214486051289

----------------- Training at steps:6 start! -----------------
-------------------------- Scene: 0 --------------------------
318 blue_1 Shoot at red_0 launch distance : 57985.865558412246 True True
Same tgt shoot
341 blue_1 Shoot at red_1 launch distance : 50023.3090162661 True True
361 red_1 Shoot at blue_1
430 blue_1 Splash :red_0
433 blue_1 Splash :red_1
WIN
blue_0 False True 561 12.038030066488414 1.4350561664884225
blue_1 False True 561 12.13803743315508 122.29740615520194
-------------------------- Scene: 0 --------------------------
308 blu